# 🎬 Manim 动画：GuessNameTree 对话与决策树同步构建

In [ ]:
from manim import *
config.background_color = WHITE
config.pixel_height = 720
config.pixel_width = 1280
config.frame_height = 6.0
config.frame_width = 10.0

## ✨ 动画类定义：GuessNameTree

In [ ]:
class GuessNameTree(Scene):
    def construct(self):
        title = Text("游戏-猜四大名著里的名字", font_size=40, weight=BOLD).to_edge(UP)
        title.shift(UP * 1.5)
        self.play(title.animate.shift(DOWN * 1.5), run_time=0.8)
        self.wait(0.7)

        dialog = [
            ("A", "是男的还是女的？", "性别", "男的", "女的"),
            ("B", "男的"),
            ("A", "是西游记的还是三国的？", "出处", "西游记", "三国"),
            ("B", "西游记的"),
            ("A", "是师徒四人中的还是妖怪？", "角色", "师徒四人", "妖怪"),
            ("B", "师徒四人中的"),
            ("A", "是不是孙悟空？", "名字", "孙悟空", "其他"),
            ("B", "√")
        ]

        color_a = "#95ec69"
        color_b = "#e5e5e5"
        avatar_radius = 0.25
        dialog_mobs = []
        y_start = 1.8
        y_gap = 0.7

        for i, entry in enumerate(dialog):
            who = entry[0]
            text = entry[1]
            y = y_start - len(dialog_mobs) * y_gap
            txt = Text(text, font_size=24, color=BLACK, weight=BOLD)
            bubble_width = txt.width + 1.2
            bubble_height = 0.8

            if who == "A":
                avatar = VGroup(
                    Circle(radius=avatar_radius, color=BLUE, fill_opacity=1),
                    Text("A", font_size=20, weight=BOLD, color=BLACK).move_to(ORIGIN)
                ).move_to(LEFT * 5.5 + UP * y)
                bubble = RoundedRectangle(corner_radius=0.3, width=bubble_width, height=bubble_height,
                                          color=color_a, fill_color=color_a, fill_opacity=1).next_to(avatar, RIGHT, buff=0.3)
                triangle = Polygon([0, 0, 0], [0.18, 0.09, 0], [0.18, -0.09, 0],
                                   color=color_a, fill_color=color_a, fill_opacity=1).next_to(bubble, LEFT, buff=-0.01)
                txt.move_to(bubble)
                group = VGroup(avatar, bubble, triangle, txt)
            else:
                avatar = VGroup(
                    Circle(radius=avatar_radius, color=GREEN, fill_opacity=1),
                    Text("B", font_size=20, weight=BOLD, color=BLACK).move_to(ORIGIN)
                ).move_to(RIGHT * 5.5 + UP * y)
                bubble = RoundedRectangle(corner_radius=0.3, width=bubble_width, height=bubble_height,
                                          color=color_b, fill_color=color_b, fill_opacity=1).next_to(avatar, LEFT, buff=0.3)
                triangle = Polygon([0, 0, 0], [-0.18, 0.09, 0], [-0.18, -0.09, 0],
                                   color=color_b, fill_color=color_b, fill_opacity=1).next_to(bubble, RIGHT, buff=-0.01)
                if entry[1] == "√":
                    txt = Text("√", font_size=32, color=GREEN, weight=BOLD).move_to(bubble)
                else:
                    txt.move_to(bubble)
                group = VGroup(bubble, triangle, txt, avatar)

            dialog_mobs.append(group)
            self.play(FadeIn(group), run_time=0.5)
            self.wait(0.3)

        self.play(FadeOut(title), run_time=0.5)
        all_dialog = VGroup(*dialog_mobs)
        self.play(
            all_dialog.animate.scale(0.6).to_corner(UL).shift(DOWN * 0.5 + RIGHT * 0.5),
            run_time=1
        )
        self.wait(0.5)

        a_dialogs = [d for d in dialog if d[0] == "A"]
        b_answers = [d[1] for d in dialog if d[0] == "B"]

        tree_nodes = []
        x0 = 3.5
        y0 = 2
        y_gap = 1.6
        x_gap = 2.4

        for i, (text, label, left_text, right_text) in enumerate(
                [(d[1], d[2], d[3], d[4]) for d in a_dialogs]):

            if i == 0:
                pos = RIGHT * x0 + UP * y0
            else:
                parent = tree_nodes[i - 1]
                direction = LEFT if b_answers[i - 1] in (a_dialogs[i - 1][3], "√") else RIGHT
                pos = parent["node"].get_bottom() + direction * x_gap + DOWN * y_gap

            node = RoundedRectangle(corner_radius=0.2, width=2.2, height=0.9,
                                    color=BLUE, fill_color=WHITE, fill_opacity=1)
            node_text = Text(label, font_size=20).move_to(node)
            node_group = VGroup(node, node_text).move_to(pos)
            self.play(FadeIn(node_group), run_time=0.5)

            if i > 0:
                parent = tree_nodes[i - 1]
                line = Line(parent["node"].get_bottom(), node_group.get_top(), buff=0.1)
                label_txt = Text(b_answers[i - 1], font_size=16).next_to(line, LEFT, buff=0.1)
                self.play(Create(line), FadeIn(label_txt), run_time=0.4)
                parent["lines"].append(line)
                parent["labels"].append(label_txt)

            tree_nodes.append({"node": node_group, "lines": [], "labels": []})

        tick = Text("√", font_size=32, color=GREEN)
        tick.next_to(tree_nodes[-1]["node"], DOWN)
        self.play(FadeIn(tick), run_time=0.5)
        self.wait(1)

## 🎥 如何渲染
```bash
manim -pql GuessNameTree.ipynb GuessNameTree
```
或使用高质量：
```bash
manim -pqh GuessNameTree.ipynb GuessNameTree
```